In [ ]:
# Import modules
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs


def get_page(url):
    '''
    A function to get a url file as lxml using BeautifulSoup
    
    Args:
        url: The url address of the page to be scrapped
    '''
    response = rq.get(url)
    parsed_page = bs(response.text, 'lxml')
    return parsed_page


def next_page(parsed_page):
    '''
     A function that gets the url for the next page from the current page
     
     Args:
         parsed_page: A parsed HTML of a webpage.
    '''
    
    pagination = parsed_page.find('div', {'class': 'pagination-wrap'})
    if pagination.find('li', {'class': 'active'}):
        new_url = pagination.find('a', {'id': 'page_next'})['href']
        return new_url
    else:
        return


def scrape_data(parsed_page, url, fuel_type):
    '''
     A function that gets the items in the given url
     
     Args:
         parse_url: url for the page to scrape
    '''
      
    url_list = [url, ]
    df_list = [] # list to contain details
    
    
    for i in range(1, 21):
        try: # Exception handling to handle the error when the url returned is 'None'
            page = get_page(url)
            url = next_page(url)
            if not url:
                break
            url_list.append(url)

        except:
            print(f'End of file. No URL returned!!!')  

    
    # iterate though products in the page to check if vehicle uses diesel
    for url in url_list:
        for i in range(1, 21):
            item_url_short = parsed_page.find('div', {'id': 'pagination_container_'+str(i)}).find('div', {'class':'product-btns'}).find_all('a', {'clas':''})[1]['href']
            item_url_full = 'https://rv.campingworld.com'+str(item_url_short)
            response_item = rq.get(item_url_full)
            parse_item = bs(response_item.text, 'lxml')
            fuel = parse_item.find('div', {'class':'tab-content'}).find_all('div', {'class':'oneSpec clearfix'})[1].find('h5').text

            # Get vehicle details if it meets gas type specification
            if fuel_type == fuel:
                vehicle_name =  parse_item.find('div', {'class':'card__title'}).find('h1', {'itemprop': 'name'}).text.split(' ', 1)[1]
                #print(vehicle_name)
                stock_number = parse_item.find('div', {'class':'stock-num-prod-details'}).text.split(' ')[2]
                status = parse_item.find('div', {'class':'product-card-line'}).find('h1', {'id': '#used-or-new'}).text
                location = parse_item.find('span', {'class':'stock-results'}).find('b').text + ', ' + list(parse_item.find('span', {'class':'stock-results'}).stripped_strings)[1]
                fuel_type = parse_item.find('div', {'class':'tab-content'}).find_all('div', {'class':'oneSpec clearfix'})[1].find('h5').text
                sleeps = parse_item.find('div', {'class':'tab-content'}).find_all('div', {'class':'oneSpec clearfix'})[3].find('h5').text
                length = parse_item.find('div', {'class':'tab-content'}).find_all('div', {'class':'oneSpec clearfix'})[2].find('h5').text[:5]
                price_low = parse_item.find('span', {'class':'price-info low-price'}).text[1:].replace(',', '')
                if int(price_low) > 300000:
                    horse_power = parse_item.find('div', {'class':'tab-content'}).find_all('div', {'class': 'oneSpec clearfix'})[6].find('h5').text
                else:
                    horse_power = ''
                horse_power = parse_item.find('div', {'class':'tab-content'}).find_all('div', {'class':'oneSpec clearfix'})
                #print(vehicle_name)

                #append details to list of fictionaries
                df_list.append({'vehicle_name': vehicle_name,
                                'stock_number': int(stock_number),
                                'status': status,
                                'location': location,
                                'fuel_type': fuel_type,
                                'sleeps': int(sleeps),
                                'length': float(length),
                                'price_low ($)': float(price_low)})
                
    # transform extracted data into a dataframe
    df = pd.DataFrame(df_list, columns = ['vehicle_name', 'stock_number', 'status', 'location','fuel_type', 'sleeps',
                                                  'length', 'price_low ($)', 'horse_power'])
    return df


url = 'https://rv.campingworld.com/rvclass/motorhome-rvs'
page = get_page(url)
check = scrape_data(page, url, 'Diesel')
check

End of file. No URL returned!!!
End of file. No URL returned!!!
End of file. No URL returned!!!
End of file. No URL returned!!!
End of file. No URL returned!!!
End of file. No URL returned!!!
End of file. No URL returned!!!
End of file. No URL returned!!!


In [3]:
# url = 'https://rv.campingworld.com/rvclass/motorhome-rvs'
# page = get_page(url)
# check = scrape_data(page, url, 'Diesel')
# check

In [41]:
# Import modules
import re
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs


# def get_page(url):
#     '''
#     A function to get the next url file as lxml using BeautifulSoup
    
#     Args:
#         url: The url address of the page to be scrapped
#     '''
#     response = rq.get(url)
#     parse_page = bs(response.text, 'lxml')
#     return parse_page


# def next_page(parse_page):
#     '''
#      A function that gets the url for the next page
     
#      Args:
#          file: A parsed HTML of a webpage.
#     '''
    
#     pagination = parse_page.find('div', {'class': 'pagination-wrap'})
#     if pagination.find('li', {'class': 'active'}):
#         new_url = pagination.find('a', {'id': 'page_next'})['href']
#         return new_url
#     else:
#         return


def scrape_data(url, fuel_type):
    '''
     A function that gets the items in the given url
     
     Args:
         parse_url: url for the page to scrape
    '''
      
    #url_list = [url, ]
    df_list = [] # list to contain details

    for i in range(1, 2):
        page_url = url + '?page=' + str(i)  
        response = rq.get(url)
        parse_page = bs(response.text, 'lxml')
        search_body = parse_page.find('div', {'id': 'content-search__body'}) # Get the section with container frames
        container_frames = re.findall('id="pagination_container_\w+"', str(search_body)) # regular expression to find all item containers
        
        # loop through each container item and get required details
        for i in range(1, len(container_frames) + 1, 1):
            item_url_short = parse_page.find('div', {'id': 'pagination_container_'+str(i)}).find('div', {'class':'product-btns'}).find_all('a', {'clas':''})[1]['href']
            item_url_full = 'https://rv.campingworld.com' + str(item_url_short) # url to item details
            response_item = rq.get(item_url_full)
            parse_item = bs(response_item.text, 'lxml')
            tab_content = parse_item.find('div', {'class':'tab-content'}).find_all('div', {'class':'oneSpec clearfix'}) # Get tab container with specifications
            specifications_1 = [spec.find('h4').text for spec in tab_content] # list cont
            specifications_2 = [spec.find('h5').text for spec in tab_content] # list containing specifications of each item
            specifications = dict(zip(specifications_1, specifications_2))
            
            # Get vehicle details if it meets gas type specification
            if fuel_type == specifications['FUEL TYPE']:
                vehicle_name =  parse_item.find('div', {'class':'card__title'}).find('h1', {'itemprop': 'name'}).text.split(' ', 1)[1]
                stock_number = parse_item.find('div', {'class':'stock-num-prod-details'}).text.split(' ')[2]
                status = parse_item.find('div', {'class':'product-card-line'}).find('h1', {'id': '#used-or-new'}).text
                location = parse_item.find('span', {'class':'stock-results'}).find('b').text + ', ' + list(parse_item.find('span', {'class':'stock-results'}).stripped_strings)[1]
                price_low = parse_item.find('span', {'class':'price-info low-price'}).text[1:].replace(',', '')
                if int(price_low) > 300000:
                    horse_power = specifications['HORSEPOWER']
                else:
                    horse_power = ''
                fuel = fuel_type
                sleeps = specifications['SLEEPS']
                length = specifications['LENGTH'][:5]
                

                #append extracted data to list of dictionaries
                df_list.append({'vehicle_name': vehicle_name,
                                'stock_number': int(stock_number),
                                'status': status,
                                'location': location,
                                'fuel_type': fuel_type,
                                'sleeps': int(sleeps),
                                'length': float(length),
                                'price_low ($)': float(price_low)})
            else:
                continue
    
    # transform extracted data into a data frame
    df = pd.DataFrame(df_list, columns = ['vehicle_name', 'stock_number', 'status', 'location','fuel_type', 'sleeps',
                                          'length', 'price_low ($)', 'horse_power'])
    return df


url = 'https://rv.campingworld.com/rvclass/motorhome-rvs'
check = scrape_data(url, 'Gas')
check

In [45]:
# url = 'https://rv.campingworld.com/rvclass/motorhome-rvs'
# for i in range(1, 51):
#     url_ = url + '?page=' + str(i)
#     print(url_)